# Pandas

*Chris Wu*  
*2016年12月*

Pandas 在处理数据方面提供了比 NumPy 更丰富的功能。NumPy 更多地用于数学计算，而 Pandas 则可以胜任许多数据分析的工作。在阅读 Pandas 之前，你最好有一些 NumPy 的基础概念（可以点这里：[NumPy](https://github.com/wklchris/Note-by-Jupyter/blob/master/Python/NumPy.ipynb)）。不过即便没有，你也可以不太困难地理解本笔记。

本笔记全篇默认在代码前都加载了以下几行：

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 基本数据类型

pandas 的基本数据类型有两种：Series 和 DataFrame：

In [2]:
s = Series([1 , 2, np.nan, 4])  # 带索引的列向量
s

0    1.0
1    2.0
2    NaN
3    4.0
dtype: float64

In [3]:
df = DataFrame({ 'states': ['A', 'B'], 'year':[2001, 2002]})  # 带列名称和行索引的矩阵
df

,states,year
0,A,2001
1,B,2002


你可以在创建 DataFrame 时指定它的行名称和列名称：

In [4]:
arr = np.arange(1, 19).reshape([6,3])
df = DataFrame(arr, index=list('ABCDEF'), columns=list('XYZ'))
df

,X,Y,Z
A,1,2,3
B,4,5,6
C,7,8,9
D,10,11,12
E,13,14,15
F,16,17,18


## 预览数据

通过 df.head/df.tail 可以预览数据的前几行/末几行。比如：

In [5]:
df.head(3)  # 默认前5行。传入参数3表示前三行。同理df.tail

,X,Y,Z
A,1,2,3
B,4,5,6
C,7,8,9


## 行标签（索引）和列标签

通过 index/columns/values 可以查看 DataFrame 的索引、列名或者数值。直接赋值可以更改它们。

In [6]:
df.index

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [7]:
df.columns

Index(['X', 'Y', 'Z'], dtype='object')

In [8]:
df.values  # 返回一个 np.array 

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

## 基本操作

### 尺寸、复制、转置

像 np.array 一样，它有 shape 这个成员，存储它的尺寸：

In [9]:
df.shape

(6, 3)

同样需要 copy() 命令来完成深度复制，否则在新变量上的更改都会反映到原变量中。

In [10]:
df2 = df.copy()

转置仍然是这样的语法：

In [11]:
df.T

,A,B,C,D,E,F
X,1,4,7,10,13,16
Y,2,5,8,11,14,17
Z,3,6,9,12,15,18


### 排序

按照轴索引排序，使用 df.sort_index() 命令：

In [12]:
df = DataFrame({'B':[0, 7, 3], 'A':[2, 4, 5]}, index=list('PCD'), dtype='float')
df

,A,B
P,2.0,0.0
C,4.0,7.0
D,5.0,3.0


In [13]:
df.sort_index()  # 默认排序行，升序（C<D<P）

,A,B
C,4.0,7.0
D,5.0,3.0
P,2.0,0.0


In [14]:
df.sort_index(ascending=False)  # 降序

,A,B
P,2.0,0.0
D,5.0,3.0
C,4.0,7.0


In [15]:
df.sort_index(axis=1, ascending=False)  # 排序列，降序

,B,A
P,0.0,2.0
C,7.0,4.0
D,3.0,5.0


想要按照某列的值排序，使用 sort_values() 命令：

In [16]:
df.sort_values(by='B')  # 此时只能按照某列的值排序，不能按照某行的值排序

,A,B
P,2.0,0.0
D,5.0,3.0
C,4.0,7.0


### 切片与选取

普通的切片命令有以下几种：

- 简单选取某列：df['A'] 或者 df.A  
- 简单选取若干行： df[:2] （选取前两行） 
- 按标签选取： df.loc['P'] / df.loc[['P', 'D'], 'A']  
- 按序号选取： df.iloc[2] （第3行）/ df.iloc[:2, :1]  

In [17]:
df

,A,B
P,2.0,0.0
C,4.0,7.0
D,5.0,3.0


In [18]:
df['A']  # 第 A 列

P    2.0
C    4.0
D    5.0
Name: A, dtype: float64

In [19]:
df[:2]  # 前两行

,A,B
P,2.0,0.0
C,4.0,7.0


In [20]:
df.loc['P']  # 按行索引标签

A    2.0
B    0.0
Name: P, dtype: float64

In [21]:
df.loc[['P', 'D'], 'A']  # 按列标签。如果选中单个元素，可以用 df.at[]

P    2.0
D    5.0
Name: A, dtype: float64

In [22]:
df.iloc[2]  # 按序号，第3行。如果选中单个元素，可以用 df.iat[]

A    5.0
B    3.0
Name: D, dtype: float64

In [23]:
df.iloc[:2, :1]  # 前两行，前一列

,A
P,2.0
C,4.0


最后再来说布尔型选取：

In [24]:
df[df.B > 0]  # 筛去列B中不大于零元素对应的行

,A,B
C,4.0,7.0
D,5.0,3.0


In [25]:
df[df > 0]  # 对整个 DataFrame 进行判断。不符合条件的位置，返回结果 np.nan

,A,B
P,2.0,NaN
C,4.0,7.0
D,5.0,3.0


In [26]:
df[df['A'].isin([2 , 4])]  # 列 A 中所有在列表[2, 4]中的元素，对应的行

,A,B
P,2.0,0.0
C,4.0,7.0


### 更改数值

你可以直接指定一列进行更改：

In [27]:
df['B'] = [0., 8, 4]
df

,A,B
P,2.0,0.0
C,4.0,8.0
D,5.0,4.0


也可以通过 loc/iloc 命令选取区域，或者 at/iat 命令选取单个元素：

In [28]:
df.iat[0 ,1] = np.nan  # 第1行第2列赋值为 np.nan（注意：只有float类型才能赋值成功）
df

,A,B
P,2.0,NaN
C,4.0,8.0
D,5.0,4.0


### 缺失数据处理：np.nan

三种方式的例子：

- df.dropna(how='any')  # 删除含有 NaN 的所有行  
- df.fillna(value=0)   # 将所有 NaN 用0替换
- pd.isnull(df)   # 返回一个布尔型矩阵

In [29]:
df.dropna(how='any')

,A,B
C,4.0,8.0
D,5.0,4.0


In [30]:
df.fillna(value=-2)

,A,B
P,2.0,-2.0
C,4.0,8.0
D,5.0,4.0


In [31]:
pd.isnull(df)

,A,B
P,False,True
C,False,False
D,False,False
